<p style="float: left;"><a href="_index.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="_index.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Actor Model

Actors are objects which encapsulate state and behavior, they communicate exclusively by exchanging messages which are placed into the recipient’s mailbox.


## Actors and Scala

Integrated within the Akka library:

- **Akka**: A library for distributed actors
- **Concurrency**
- **Scalability** (locally through additional threads and externally via actors on other nodes)
- **Fault tolerance**
- **Unified programming model**
- **Integrated runtime management**
- **Open-source**

## The Problem of Computation on Distributed Systems

Object-oriented programming (OOP) is a widely-accepted, familiar programming model. One of its core pillars is _encapsulation_.
Encapsulation dictates that the internal data of an object is not accessible directly from the outside; it can only be modified by invoking a set of curated methods. 
_**The object is responsible for exposing safe operations that protect the invariant nature of its encapsulated data**_.

When we analyze OOP runtime behavior, we sometimes draw a message sequence chart showing the interactions of method calls. For example:

<center>
    <img src="https://getakka.net/images/seq_chart.png"/>
</center>

In reality, a _thread_ executes all these calls, and the enforcement of invariants occurs on the same thread 
from which the method was called. Updating the diagram with the thread of execution, it looks like this:

<center>
    <img src="https://getakka.net/images/seq_chart_thread.png"/>
</center>

The significance of this clarification becomes clear when you try to model what happens with multiple threads. 
Suddenly, our neatly drawn diagram becomes inadequate. We can try to illustrate multiple threads accessing the same instance:

<center>
    <img src="https://getakka.net/images/seq_chart_multi_thread.png"/>
</center>

_**The encapsulation model of objects does not guarantee anything about what happens where two threads enter the same method**_. Instructions of the two invocations can be interleaved in arbitrary ways which eliminate any hope for keeping the invariants intact without some type of coordination.

The common approach to solving this problem is to add a lock around these methods. This is a very costly strategy:

* Locks seriously limit concurrency, they are very costly on modern CPU architectures, requiring heavy-lifting from the operating system to suspend the thread and restore it later.
* The caller thread is now blocked, so it cannot do any other meaningful work.
* Locks introduce a new menace: deadlocks.

Additionally, locks only really work well locally. When it comes to coordinating across multiple machines, the only alternative is distributed locks. Unfortunately, _**distributed locks are several magnitudes less efficient than local locks**_ and usually impose a hard limit on scaling out. 

_**You might thing that call stacks are a good way to operate between threds**_, yet they were designed in an era before widespread concurrent programming, _**they don’t handle asynchronous operations well because they don’t cross thread boundaries**_. When a task is delegated to another thread (typically a "background" worker), it involves the "caller" placing the task in shared memory for the "worker" to pick up, allowing the caller to continue with other tasks.

<center>
    <img src="https://getakka.net/images/exception_prop.png"/>
</center>

However, this setup raises challenges: 

1. **Completion notification**: How does the "caller" know the task is complete?
2. **Exception handling**: If an error occurs, the exception goes to the worker’s handler, bypassing the caller. Without a direct way to notify the caller, the task’s failure may go unnoticed unless a side-channel, like an error code, is implemented.

These issues resemble network failures, where messages can be lost. The problem worsens if the worker thread encounters a critical bug, causing it to terminate and lose any in-progress task data. This leaves the system without a way to recover the lost task, even in local (non-networked) communication.

_**To enable effective concurrency, threads must delegate tasks efficiently without blocking**_. However, 
_**with this delegation model, traditional call stack error handling fails**_, requiring explicit error-signaling mechanisms where
failures become part of the system model. Concurrent systems must handle service faults and provide recovery mechanisms.

## How the Actor Model Meets the Needs of Concurrent, Distributed Systems

- Concurrent computation model
- Think of an actor as a fundamental unit of concurrent computation.
- Like in Object-Oriented Programming (OOP), actors hold state, but manage it through message passing rather than direct method calls.
- Actors can modify their own private state but interact with each other only via messaging, which eliminates the need for lock-based synchronization.
- Actors have a mailbox where messages arrive and are processed in the order they are received.
- How an actor responds to a message essentially defines its behavior.

An actor has the ability to:

1) Send messages to other actors,
2) Respond to received messages, and
3) Create new actors.


An important difference of passing messages instead of calling methods is that messages have no return value. And
Actors react to messages just like objects "react" to methods invoked on them. They execute independently from the senders of a message, and they react to incoming messages sequentially, at most one at a time.

<center>
    <img src="https://getakka.net/images/serialized_timeline_invariants.png"/>
</center>

### Hierarchical Structure

* Similar to an organizational hierarchy, actors in Akka naturally form tiers.
  - For example, an actor responsible for a particular function within the program may choose to break down its task into smaller, manageable segments.
  - To achieve this, it creates child actors, over which it maintains supervisory control.
* Each actor has one supervisor, the actor that initiated its creation.
* When an actor is unable to address a certain situation, it sends a failure message to its supervisor, requesting assistance.
* This recursive setup enables failure handling at the appropriate level in the hierarchy.

<center>
    <img src="https://getakka.net/images/actor_tree_supervision.png"/>
</center>

### Messages and Immutability

Messages can be any kind of object but have to be immutable. Messages are sent to an Actor using one of the following methods:

* **tell (!)** : a "fire-and-forget" approach, where a message is sent asynchronously, and the sender returns immediately without waiting for a response.

* **ask (?)** : sends a message asynchronously but returns a `Future` that represents a possible reply from the receiver.

* In each of these methods, the sender has the option to include its own `ActorRef`.
This allows the receiving actor to respond directly to the sender, as the reference to the sender is passed along with the message.

* Message ordering is guaranteed for messages sent from the same sender to a receiver.

## Let's play with Actors

In [1]:
import $ivy.`com.typesafe.akka::akka-actor:2.6.10`
import akka.actor.{Actor, ActorRef, Props}
import akka.actor.{ActorSystem, PoisonPill, Props}

Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-actor_2.13/2.6.10/akka-actor_2.13-2.6.10.pom
Downloaded https://repo1.maven.org/maven2/com/typesafe/config/1.4.0/config-1.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-java8-compat_2.13/0.9.0/scala-java8-compat_2.13-0.9.0.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-java8-compat_2.13/0.9.0/scala-java8-compat_2.13-0.9.0-sources.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/config/1.4.0/config-1.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-java8-compat_2.13/0.9.0/scala-java8-compat_2.13-0.9.0.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/config/1.4.0/config-1.4.0.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-actor_2.13/2.6.10/akka-actor_2.13-2.6.10-sources.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-actor_2.13/2.6.10/akka-actor_2.13-2.6.10.jar


import $ivy.$
import akka.actor.{Actor, ActorRef, Props}
import akka.actor.{ActorSystem, PoisonPill, Props}

In [2]:
case class Greet(name: String)
case class Praise(name: String)
case class Celebrate(name: String, age: Int)

class Talker extends Actor {
    var messages = 0
    
    def receive = {
        case Greet(name)          => greet(name)
        case Praise(name)         => praise(name)
        case Celebrate(name, age) => celebrate(name, age)
    }

    def updateMessages(n: Int) = {
        messages += 1
        println(s"Message n: $messages")
    }

    def greet(name: String) = {
        updateMessages(1)
        println(s"Hello $name")
    }

    def praise(name: String) = {
        updateMessages(1)
        println(s"$name, you're amazing")
    }

    def celebrate(name: String, age: Int) = {
        updateMessages(1)
        println(s"Here's to another $age years, $name")
    }
}

defined class Greet
defined class Praise
defined class Celebrate
defined class Talker

In [3]:
object HelloActors {
    val system = ActorSystem("HelloActors")
    val talker = system.actorOf(Props(new Talker), "talker")
    talker ! Greet("Huey")
    talker ! Praise("Dewey")
    talker ! Celebrate("Louie", 16)
    Thread.sleep(5000)
    system.terminate()
}

HelloActors

Message n: 1
Hello Huey
Message n: 2
Dewey, you're amazing
Message n: 3
Here's to another 16 years, Louie


defined object HelloActors
res3_1: HelloActors.type = ammonite.$sess.cmd3$Helper$HelloActors$@3945bd46

In [3]:
object BadHelloActors {
    val system = ActorSystem("HelloActors")
    val talker = system.actorOf(Props(new Talker), "talker")
    talker.greet()
    Thread.sleep(10000)
    system.terminate()
}

BadHelloActors

cmd4.sc:4: value greet is not a member of akka.actor.ActorRef
    talker.greet()
           ^
Compilation Failed

**NOTE:** The example below may seem misleading because this notebook is running on a single-thread container, giving the impression that everything is executing synchronously.

Let’s dive into the [actors-nightmare](https://github.com/wilberquito/actors-nightmare) repository and get it running on our PCs!

<p style="float: left;"><a href="_index.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="_index.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>